## Notebook Index
1. [Feature Store](https://app.snowflake.com/sfpscogs/rpegu_aiml/#/notebooks/ML_MODELS.DS.%2201_FeatureStore_Creation%22) 
2. Feature Reduction 👈
3. [Model Training](https://app.snowflake.com/sfpscogs/rpegu_aiml/#/notebooks/ML_MODELS.DS.%2203_Model_Training%22)
4. [Model Inference & scheduling ](https://app.snowflake.com/sfpscogs/rpegu_aiml/#/notebooks/ML_MODELS.DS.%2204_Batch_Inferencing%22)


## What this notebook does?
In this notebook we create the Dataset that will be used for modeling purpose, we also do feature reduction and remove the unwanted or redundant features before doing the model training

#### Feature Reduction
Feature Reduction Using Correlation and Variance Threshold

Feature reduction helps simplify models, reduce overfitting, and improve computational efficiency.

Variance Threshold: Removes features with low variance, which carry little information for prediction. For example, if a feature’s value is almost constant across samples, it contributes minimally to model learning.

Correlation Analysis: Identifies highly correlated features (multicollinearity). Strongly correlated features (e.g., correlation > 0.8) provide redundant information. Retaining only one from such groups helps prevent model instability and reduces complexity.

Best Practice: Apply variance thresholding first to remove uninformative features. Then perform correlation analysis to eliminate redundancy among remaining features.

In [ ]:
# Import python packages
import pandas as pd

# Snowflake Feature Store
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,  CreationMode)
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType, DoubleType, StringType
from snowflake.snowpark.functions import month,year,col,sum,when,upper, regexp_replace

from snowflake.snowpark.version import VERSION

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()



In [ ]:

#database
input_database          = 'ML_MODELS'
working_database       = 'ML_MODELS'

#schema
input_schema            = 'DS'
working_schema          = 'DS'
fs_schema               = 'FEATURE_STORE'

warehouse = 'DS_W'
snowpark_opt_warehouse  = 'SNOWPARK_OPT_WH'
session.use_warehouse(warehouse )
session.use_role('FR_SCIENTIST')
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION
# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [ ]:
tablenm =f"{working_database}.{working_schema}.DEMO_DATASET_V_1"
df = session.table(tablenm)
print(f"total columns in the dataframe :{len(df.columns)}")
print(f"total rows in the dataframe :{df.count()}")
df.show()

### Pre-Processing Steps
For the demo purposes, I will the REF_MMYY=042025 for Model Training  and REF_MMYY=052025 for validation for purposes.

**Data Type Conversion**:
* Convert all columns with DECIMAL data type to DOUBLE for consistency in numerical calculations.

**String Column Standardization**:

* Convert all string values to UPPERCASE.
* Remove spaces and special characters to ensure clean and consistent categorical values.

In [ ]:

#sdf = df


## get all columns with stringType= type
excluded = ['MEMBER', 'TARGET','REF_MMYY']
features = [col for col in df.columns if col not in excluded]

cat_cols = [field.name for field in df.schema.fields if isinstance(field.datatype, StringType)]
cat_cols=[col for col in cat_cols if col != excluded] 
cat_cols
## Making sure the string column has values in UPPER Case, no space  or special character

# Apply transformations to upper and remove space
def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))


for col in cat_cols:
    sdf = df.with_column(col, fix_values(col))


##NUMERICALCOL
## converting the decimal into double

decimal_col = [field.name for field in sdf.schema.fields if isinstance(field.datatype, DecimalType)]
for colname in decimal_col: sdf = sdf.with_column(colname,sdf[colname].cast(DoubleType()))

 

In [ ]:
sdf.show()
###sdf.columns

### Missing Value Handling:

* Apply KNN Imputer for missing values in numeric columns.

* For string columns, handle nulls by either imputing with a default value or marking them explicitly as 'UNKNOWN'.

In [ ]:
from snowflake.ml.modeling.impute import KNNImputer
Missing_data_col=['FEATURE_0', 'FEATURE_10', 'FEATURE_50']
#sdf[Missing_data_col].show()
## using the KNN imputer natively inside Snowflake Ml to handle missing Data 

# Initialize KNNImputer
imputer = KNNImputer(input_cols=Missing_data_col,
                     output_cols=Missing_data_col,
                     n_neighbors=3)

# Fit and transform
sdf = imputer.fit(sdf).transform(sdf)

## Categorical col
# Create dictionary to fill nulls with unknown for those columns
fill_values = {col_name: 'UNKNOWN' for col_name in cat_cols}
# Apply fillna to categorical col 
sdf = sdf.fillna(fill_values)

In [ ]:
session.use_warehouse(snowpark_opt_warehouse)

from snowflake.snowpark import DataFrame
#from snowflake.snowpark.functions import var_pop

## get all columns with stringType= type
excluded = ['MEMBER_ID', 'TARGET','REF_MMYYYY','CAT_1','CAT_2','CAT_3','CAT_4','CAT_5']
num_cols = [col for col in sdf.columns if col not in excluded]

session.use_warehouse('SNOWPARK_OPT_W')
print(f'number of features before the variance threshold {len(num_cols)}')

# get the
variance_df = sdf.select([F.var_pop(F.col(c)).alias(c) for c in num_cols])

variance_df = variance_df.to_pandas()
cols_below_threshold  = variance_df.columns[(variance_df  < 0.1).all()]
print( f" total cols having variance threshold less than 0.1  is {len(cols_below_threshold)}")

sdf=sdf.drop(*cols_below_threshold )

print(f'number of features after applying the variance threshold  {len(cols_below_threshold)}')


In [ ]:

from snowflake.ml.modeling.metrics.correlation import correlation


def snf_correlation_thresholder(df, features, corr_threshold: float):
    assert 0 < corr_threshold <= 1, "Correlation threshold must be in range (0, 1]."
    
    corr_features = set()
    corr_matrix = correlation(df=sdf)

    # Compute pairwise correlations directly in Snowpark
    for i in range(len(features)):
        for j in range(i + 1, len(features)):
            if (abs(corr_matrix.iloc[i][j])) >=  corr_threshold:
            #col1, col2 = features[i], features[j]
            #corr_value = df.select(corr(col(col1), col(col2)).alias('corr')).collect()[0]['CORR']
            
           # if corr_value is not None and abs(corr_value) >= corr_threshold:
                # Mark the second feature for removal to avoid keeping highly correlated pairs
                #corr_features.add(col2)
                corr_features.add(features[j])
    
    # Drop correlated features if any
    if corr_features:
        df = df.drop(*corr_features)
        
    return df

In [ ]:
excluded = ['MEMBER_ID', 'TARGET','REF_MMYYYY','CAT_1','CAT_2','CAT_3','CAT_4','CAT_5']
num_cols = [col for col in sdf.columns if col not in excluded]
print(f'number of features after applying the variance threshold  {len(sdf.columns)}')

sdf = snf_correlation_thresholder(sdf, num_cols, 0.8)
print(f'number of features after applying the variance threshold  {len(sdf.columns)}')

In [ ]:
--ALTER DATASET Reduced_Dataset DROP VERSION 'V_1';

In [ ]:
from snowflake.ml import dataset
# Materialize DataFrame contents into a Dataset
ds = dataset.create_from_dataframe(
    session,
    "Reduced_Dataset",
    "V_1",
    input_dataframe=sdf)